In [2]:
import os
# import PyPDF2
from openai import OpenAI
from tqdm import tqdm
from time import sleep
import pandas as pd
# from pdfminer.high_level import extract_text
import re
import requests
import json
from pdfplumber import open as pdf_open
from datetime import datetime


def patch_read_filename(filePath):
    """
    获取目标路径的文件名称
    """
    nameList = os.listdir(filePath)
    names= globals()
    count=0

    # 初始化一个空列表来存储文件名
    file_list = []

    # 遍历文件夹
    for filename in os.listdir(filePath):
        file_list.append(filename)

    # 输出文件名列表
    return file_list


# 读取 PDF 文档
def read_pdf(file_path):
    with pdf_open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text


# 调用 DeepSeek AI 模型
def talk_base_AI(API_KEY, model, role, question):
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key=API_KEY, # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    # 调用AI 模型
    completion = client.chat.completions.create(
        model=model, 
        messages=[
            {'role': 'system', 'content': role},
            {'role': 'user', 'content': question}
            ],
        timeout=1000
    )

    return completion.choices[0].message.content

def talk_file_AI(API_KEY, model, role, question,pdf_text,requirements):
    question = f'请参考以下内容：{pdf_text}\n question \n 要求:{requirements}'
    return talk_base_AI(API_KEY, model, role, question)


################################################################
# 配置 API 密钥和 URL
# API_URL = 'https://dashscope.aliyuncs.com/api/v1/'
API_KEY = 'sk-4e28a0f9c9064576895552d2bb361f48'
model="deepseek-r1" # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models

# 配置文件路径
file_path = r"C:\Users\Thomas P Gao\Documents\personal\VSC\VSC_work\work_documents\PWC\Audit Guide" # 请用自己的地址替换
testlist =patch_read_filename(file_path)


ModuleNotFoundError: No module named 'pdfplumber'

In [ ]:
#-----------TASK1：PDF文件内容提取----------------
# 设置ai的角色
role = '你是一名审计专家'
question = '生成markdown格式的笔记。'
requirements = "1.请不要总结/增删文字,如果是冗余空格则可以删除；2.选择合适的行作为一级标题(如3001)、二级标题(如3001.1),以此类推;3.如果遇到流程图或者表格,请使用mermaid或者markdown语法进行描述。4.由于有些文本出现了乱序,可能存在的情况是3001.1的内容调换到了3001.3后面,此时需要你按标题层级对内容顺序小作调整。"


In [ ]:
# 遍历PDF文档，得到文件名testlist; 读取pdf的内容
file_count = 0
for pdffile in testlist:
    if pdffile.endswith('.pdf'):
      file_count += 1
      if file_count == 1:
        print(pdffile)
        pdf_text = read_pdf(file_path+"/"+pdffile)
        if len(pdf_text) >=  57344:
            print(f'PDF文件{pdffile}的内容长度为{len(pdf_text)}，超过{model}模型的限制，正在分批处理...')
            # 按章节标题分块
            chunks = []
            current_chunk = []
            for index,line in enumerate(pdf_text.split('\n')) : # 如果分割
                if re.match(r'^\d+(?:\.\d+)*',line):
                    if current_chunk:
                        chunks.append('\n'.join(current_chunk))
                        current_chunk = []
                elif  index < 50:
                    print(line)
                    # if index == 30:
                    #     break
                current_chunk.append(line)
            if current_chunk:
                chunks.append('\n'.join(current_chunk))
            print(len(chunks))
            
            
        else:
            pass

2024-pwc-audit-guide--mainland-china--hong-kong_3000-understand-the-entity-and-its-environment.pdf
PDF文件2024-pwc-audit-guide--mainland-china--hong-kong_3000-understand-the-entity-and-its-environment.pdf的内容长度为1301517，超过deepseek-v3模型的限制，正在分批处理...
Global PwC Guidance
This subsection addresses the ISA requirements and related guidance on risk assessment procedures and related activities. PwC Audit 3100 provides guidance in the following
specific areas:
Activities
ISA 315 (Revised 2019) – Identifying and Assessing the Risks of Material Misstatement
ISA Objective
The objective of the auditor is to identify and assess the risks of material misstatement, whether due to fraud or error, at the financial statement and assertion levels thereby
providing a basis for designing and implementing responses to the assessed risks of material misstatement.
ISA 315 (Revised 2019) – Identifying and Assessing the Risks of Material Misstatement
ISA Requirements
Risk Assessment Procedures and Related Activitie

In [1]:
content = ''
for i, chunk in enumerate(chunks):
    # 先只运行第一块
    # if i in range(3) and i < 6:
    if i == 2:
    # if i = 1:
        print(f'正在处理第{i+1}块,长度为{len(chunk)}...')
        print(chunk)
        current_time = datetime.now()
        current_content = talk_file_AI(API_KEY, model, role, question,chunk,requirements)
        content =content+"\n"+ current_content
        cost_time = datetime.now() - current_time
        print(f'第{i+1}块处理完成，耗时{cost_time.total_seconds()}')
# 创建并写入到文件
with open(r"C:\Users\Thomas P Gao\Documents\personal\VSC\VSC_work\work_documents\PWC\Audit Guide\output2.md", "w", encoding="utf-8") as file:
    file.write(content)

print(f"文件已成功创建并写入")

NameError: name 'chunks' is not defined